# AI(t)HistChronicle - An Engine for Alternative History

## Info: AI(t)HistChronicle

### Aim
- Based on the user input, the script creates a response drawing an alternative version of historical events

### AI Capabilities used:
- RAG (Retrieval Augmented Generation) through feeding in background information from Wikipedia and news articles from Wikinews
- Controlled generation by specifying, for instance, the role the model should take and the output type the response should have

### Author
Dr. Patrick Thiel

### NOTE:
This was original a Kaggle notebook. Some things might be different if run locally (e.g. the API key handling).

## Setup

In [ ]:
# installion (if required)
# !pip install groq
# !pip install wikipedia

In [22]:
from groq import Groq
import re
import textwrap
from kaggle_secrets import UserSecretsClient
import wikipedia
import requests

In [23]:
# Retrieve API from secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("deepseek")

In [24]:
# initialize client for DeepSeek model (using API key)
client = Groq(api_key = api_key)

## Specification of Input

- Defines which counterfactural world is created

In [25]:
# inputs
event = "Assassination of John F Kennedy"
scenario = "What if JFK was not murdered?"
output_type = "news article"
role = "journalist"
additional_instructions = "Be dramatic. You are working for The Sun." # In case you want to give the text a specific twist. If not, leave empty!

## API Connection to Wikipedia

In [26]:
# retrieve background information from Wikipedia
def getting_background_wikipedia(event: str):
    summary = wikipedia.summary(event, sentences = 50)
    return summary

## Getting news articles (Wikinews)

In [50]:
def getting_wikinews(event: str):
    # look for news
    url = "https://en.wikinews.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": event,
    }
    response = requests.get(url, params = params).json()
    search_results = response.get("query", {}).get("search", [])
    
    if not search_results:
        return None

    articles_list = list()
    for element in search_results[1:5]: # NOTE: set a limit on number of articles due to the restriction of allowed processed input tokens in the model
        # Get title of article
        title = element["title"]
        
        # Get page content
        page_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "explaintext": True,
            "titles": title,
        }
        
        page_response = requests.get(url, params = page_params).json()
        pages = page_response["query"]["pages"]
        page_content = next(iter(pages.values()))["extract"]

        articles_list.append({"title": title, "content": page_content})

    return articles_list

## Text Generation

In [53]:
def generating_text(event: str, scenario: str, output_type: str):
    prompt = f"""
        Based on the following Wikipedia summary about the {event}:
        {getting_background_wikipedia(event)}
        Further, here some relevant news article from the past: {getting_wikinews(event)}
        Take on the role {role} in 2025. Write a {output_type}.
        Imagining that the following event in history had a different outcome: {event}.
        Consider the following scenario: {scenario} as counterfactual scenario.
        {additional_instructions}
        Write it not conditional. Write in active style such that you could think it really happened like this
    """

    completion = client.chat.completions.create(
        model = "deepseek-r1-distill-llama-70b",
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = 0.9,
        max_completion_tokens = 2000,
        stream = True
    )

    return completion

response = generating_text(event = event, scenario = scenario, output_type = output_type)

## Extracting the response

- The model output consists of reasoning by the model about what to write and the actual output. We are only interested in the counterfactual world, hence the generated text.

In [54]:
def extracting_response(response):
    response_text = ""
    for chunk in response:
        text = chunk.choices[0].delta.content or ""
        response_text += text

    return response_text

text = extracting_response(response)

## Adding some styling to the text

In [55]:
def formatting_text(text):
    raw_text = re.search(r'</think>(.*)', text, re.DOTALL).group(1).strip()
    lines = raw_text.split('\n')
    formatted_article = []

    # Wrap settings
    wrapper = textwrap.TextWrapper(width=90)

    # Detect and format special parts
    for line in lines:
        line = line.strip()
        if line.startswith("**Date:"):
            formatted_article.append(f"\n📅 {line.replace("**", "")}")
        elif line.startswith("**By:"):
            formatted_article.append(f"✍️  {line.replace("**", "")}\n")
        elif line.startswith("---"):
            continue  # skip the horizontal line
        elif line.startswith("**") and line.endswith("**"):
            title = line.replace("**", "")
            formatted_article.append(f"\n📰 {title}\n" + "-" * len(title))
        elif line.startswith('"') or line.startswith("“"):
            # Quote handling
            formatted_article.append("\n🗨️  " + wrapper.fill(line))
        else:
            formatted_article.append(wrapper.fill(line))
    
    # Join the final result
    formatted_output = "\n\n".join(formatted_article)
    
    # Print or use it
    return formatted_output

In [56]:
final_text = formatting_text(text)
print(final_text)


📰 JFK SURVIVES DALLAS SHOT: A PRESIDENT'S MIRACLE AND A NATION REBORN
-------------------------------------------------------------------



*Dallas, TX — November 22, 1963* — In a stunning turn of events that will be etched into
the annals of American history, President John F. Kennedy has survived the assassination
attempt in Dallas, Texas. The nation, left breathless and in disbelief, is grappling with
the magnitude of what could have been—and what now can be.



At approximately 12:30 p.m. Central Standard Time, as Kennedy’s motorcade made its way
through Dealey Plaza, shots rang out from the sixth floor of the Texas School Book
Depository. The President was struck in the neck, but in a miraculous twist, the wound was
not fatal. Texas Governor John Connally, who was riding alongside Kennedy, was also
injured but is expected to recover.



Secret Service agents acted with lightning speed, rushing the presidential limousine to
Parkland Memorial Hospital. Doctors worked tirelessly to